In [60]:
import pandas as pd
from sklearn.model_selection import train_test_split

stores_total_train = pd.read_csv("data/stores_train.csv")
stores_test = pd.read_csv("data/stores_test.csv")
stores_extra = pd.read_csv("data/stores_extra.csv")
stores_train, stores_val = train_test_split(stores_total_train, test_size=0.2, random_state=0)

In [61]:
from utils import create_geographical_columns, split_plaace_cat
geo_df = pd.concat([ stores_total_train, stores_extra , stores_test])
geo_df = geo_df[['grunnkrets_id', 'store_id', 'plaace_hierarchy_id']]
geo_df = create_geographical_columns(geo_df)
geo_df = split_plaace_cat(geo_df)
results_df = geo_df.copy()
geo_df['count'] = 1
geo_df


,grunnkrets_id,store_id,plaace_hierarchy_id,fylke,kommune,delomrade,plaace_cat_1,plaace_cat_2,plaace_cat_3,plaace_cat_4,count
0,6020303,983540538-974187930-44774,1.1.1.0,6,602,60203,1,1.1,1.1.1,1.1.1.0,1
1,3010306,987074191-973117734-44755,1.1.1.0,3,301,30103,1,1.1,1.1.1,1.1.1.0,1
2,6050102,984890265-981157303-64491,1.1.1.0,6,605,60501,1,1.1,1.1.1,1.1.1.0,1
3,18040102,914057442-992924179-126912,1.1.1.0,18,1804,180401,1,1.1,1.1.1,1.1.1.0,1
4,16017414,913018583-913063538-668469,1.1.1.0,16,1601,160174,1,1.1,1.1.1,1.1.1.0,1
...,...,...,...,...,...,...,...,...,...,...,...
8572,11490601,917323003-917383529-844309,2.8.11.2,11,1149,114906,2,2.8,2.8.1,2.8.11.2,1
8573,11030901,917353379-917411824-845904,2.8.11.2,11,1103,110309,2,2.8,2.8.1,2.8.11.2,1
8574,2270103,917072302-917089248-833647,2.8.11.2,2,227,22701,2,2.8,2.8.1,2.8.11.2,1
8575,12470701,916960557-916993161-829908,2.8.11.2,12,1247,124707,2,2.8,2.8.1,2.8.11.2,1


In [62]:
from constants import *
for level in levels:
    for cat in plaace_cols:
        grouped = geo_df.groupby([ level , cat]).sum()['count'].to_frame()
        grouped['count']
        grouped.rename(columns= {'count':f'{level}.{cat}_count'}, inplace=True)
        grouped.to_csv(path_or_buf=f"temp_data/store_nums_{level}_{cat}.csv", index=True)

In [69]:
results_df.dtypes

grunnkrets_id           int64
store_id               object
plaace_hierarchy_id    object
fylke                   int64
kommune                 int64
delomrade               int64
plaace_cat_1           object
plaace_cat_2           object
plaace_cat_3           object
plaace_cat_4           object
dtype: object

In [70]:
pd.read_csv("temp_data/store_nums_delomrade_plaace_cat_2.csv").dtypes

delomrade                         int64
plaace_cat_2                    float64
delomrade.plaace_cat_2_count      int64
dtype: object

In [71]:
for level in levels:
    for cat in plaace_cols:
        print(level,  cat)
        csv_name = f"temp_data/store_nums_{level}_{cat}.csv" 
        store_counts_df = pd.read_csv(csv_name, dtype={cat: object})
        results_df = pd.merge(results_df, store_counts_df, how='left', on=[level, cat])

grunnkrets_id plaace_cat_1
grunnkrets_id plaace_cat_2
grunnkrets_id plaace_cat_3
grunnkrets_id plaace_cat_4
delomrade plaace_cat_1
delomrade plaace_cat_2
delomrade plaace_cat_3
delomrade plaace_cat_4
kommune plaace_cat_1
kommune plaace_cat_2
kommune plaace_cat_3
kommune plaace_cat_4
fylke plaace_cat_1
fylke plaace_cat_2
fylke plaace_cat_3
fylke plaace_cat_4


In [72]:
results_df

,grunnkrets_id,store_id,plaace_hierarchy_id,fylke,kommune,delomrade,plaace_cat_1,plaace_cat_2,plaace_cat_3,plaace_cat_4,grunnkrets_id.plaace_cat_1_count,grunnkrets_id.plaace_cat_2_count,grunnkrets_id.plaace_cat_3_count,grunnkrets_id.plaace_cat_4_count,delomrade.plaace_cat_1_count,delomrade.plaace_cat_2_count,delomrade.plaace_cat_3_count,delomrade.plaace_cat_4_count,kommune.plaace_cat_1_count,kommune.plaace_cat_2_count,kommune.plaace_cat_3_count,kommune.plaace_cat_4_count,fylke.plaace_cat_1_count,fylke.plaace_cat_2_count,fylke.plaace_cat_3_count,fylke.plaace_cat_4_count
0,6020303,983540538-974187930-44774,1.1.1.0,6,602,60203,1,1.1,1.1.1,1.1.1.0,42,27,1,1,70,46,1,1,171,122,3,3,572,417,9,9
1,3010306,987074191-973117734-44755,1.1.1.0,3,301,30103,1,1.1,1.1.1,1.1.1.0,103,74,2,2,242,175,3,3,2186,1592,35,35,2186,1592,35,35
2,6050102,984890265-981157303-64491,1.1.1.0,6,605,60501,1,1.1,1.1.1,1.1.1.0,8,6,1,1,45,31,1,1,57,40,1,1,572,417,9,9
3,18040102,914057442-992924179-126912,1.1.1.0,18,1804,180401,1,1.1,1.1.1,1.1.1.0,34,21,1,1,69,51,1,1,117,85,2,2,500,369,3,3
4,16017414,913018583-913063538-668469,1.1.1.0,16,1601,160174,1,1.1,1.1.1,1.1.1.0,14,6,2,2,27,16,2,2,471,341,7,7,665,487,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50115,11490601,917323003-917383529-844309,2.8.11.2,11,1149,114906,2,2.8,2.8.1,2.8.11.2,1,1,1,1,36,7,6,1,234,49,35,1,2509,527,417,5
50116,11030901,917353379-917411824-845904,2.8.11.2,11,1103,110309,2,2.8,2.8.1,2.8.11.2,5,1,1,1,13,5,4,1,705,134,105,1,2509,527,417,5
50117,2270103,917072302-917089248-833647,2.8.11.2,2,227,22701,2,2.8,2.8.1,2.8.11.2,13,5,4,1,14,6,5,1,24,11,8,1,3041,571,412,5
50118,12470701,916960557-916993161-829908,2.8.11.2,12,1247,124707,2,2.8,2.8.1,2.8.11.2,6,2,2,1,10,4,4,1,101,18,16,1,2682,621,494,2
